# 3D Segmentation with UNet

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/3d_segmentation/unet_segmentation_3d_ignite.ipynb)

## Setup environment

In [1]:
from glob import glob
import os
import sys
import shutil
import numpy as np

## Setup data directory

In [2]:
#data_dir = "/content/drive/MyDrive/unet-with-monai/dataset/prostate"
raw_data_dir = "./dataset/prostate"
new_data_dir = "./dataset/working_data"
dataset_format_config = np.load("./dataset_format_config.npy", allow_pickle=True).item()
print(dataset_format_config)
expert_ids = np.array([1, 2, 3, 4, 5, 6])

{'training': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42]), 'validation': array([43, 44, 45, 46, 47, 48]), 'test': array([49, 50, 51, 52, 53, 54, 55]), 'shuffle_id': False}


In [3]:
# copy raw data to the working dir
def dir_copy(src_dir, dst_dir, case_name, shuffle):
    src_case_dir = os.path.join(src_dir, case_name)
    dst_case_dir = os.path.join(dst_dir, case_name)
    shutil.copytree(src_case_dir, dst_case_dir, dirs_exist_ok=True)
    print(f"src_case_dir = {src_case_dir}")
    print(f"dst_case_dir = {dst_case_dir}")
    print()
    if shuffle:
        np.random.shuffle(expert_ids)
        print(expert_ids)
        for seg in os.listdir(dst_case_dir):
            old_fullname = os.path.join(dst_case_dir, seg)
            for i in range(6):
                if f"seg{i+1:02d}" in seg:
                    new_fullname = old_fullname.replace(f"seg{i+1:02d}.nii.gz", f"tmp_seg{expert_ids[i]:02d}.nii.gz")
                    os.rename(old_fullname, new_fullname)
                    print(f"old_full_name: {old_fullname}")
                    print(f"new_full_name: {new_fullname.replace('tmp_', '')}")
                    break
        for seg in os.listdir(dst_case_dir):
            fullname = os.path.join(dst_case_dir, seg)
            os.rename(fullname, fullname.replace('tmp_', ''))
    print('--'*10)

## Setup data
First generate the proper file tree

In [4]:
raw_training_dir = os.path.join(raw_data_dir, 'Training')
raw_validation_dir = os.path.join(raw_data_dir, 'Validation')

# clear working dir first
shutil.rmtree(new_data_dir, ignore_errors=True)

new_training_dir = os.path.join(new_data_dir, 'Training')
new_validation_dir = os.path.join(new_data_dir, 'Validation')
new_test_dir = os.path.join(new_data_dir, 'Test')
idx = 1

# setup the data form
training = dataset_format_config['training']
validation = dataset_format_config['validation']
test = dataset_format_config['test']
shuffle = dataset_format_config['shuffle_id']

In [5]:
# extract training data
for train_id in training:
    case_name = f"case{train_id:02d}"
    if case_name in os.listdir(raw_training_dir):
        src_dir = raw_training_dir
    elif case_name in os.listdir(raw_validation_dir):
        src_dir = raw_validation_dir
    else:
        raise ValueError
    dir_copy(src_dir=src_dir, dst_dir=new_training_dir, case_name=case_name, shuffle=shuffle)

# extract validation data
for val_id in validation:
    case_name = f"case{val_id:02d}"
    if case_name in os.listdir(raw_training_dir):
        src_dir = raw_training_dir
    elif case_name in os.listdir(raw_validation_dir):
        src_dir = raw_validation_dir
    else:
        raise ValueError
    dir_copy(src_dir=src_dir, dst_dir=new_validation_dir, case_name=case_name, shuffle=shuffle)

# extract test data
for test_id in test:
    case_name = f"case{test_id:02d}"
    if case_name in os.listdir(raw_training_dir):
        src_dir = raw_training_dir
    elif case_name in os.listdir(raw_validation_dir):
        src_dir = raw_validation_dir
    else:
        raise ValueError
    dir_copy(src_dir=src_dir, dst_dir=new_test_dir, case_name=case_name, shuffle=False)

src_case_dir = ./dataset/prostate/Training/case01
dst_case_dir = ./dataset/working_data/Training/case01

--------------------
src_case_dir = ./dataset/prostate/Training/case02
dst_case_dir = ./dataset/working_data/Training/case02

--------------------
src_case_dir = ./dataset/prostate/Training/case03
dst_case_dir = ./dataset/working_data/Training/case03

--------------------
src_case_dir = ./dataset/prostate/Training/case04
dst_case_dir = ./dataset/working_data/Training/case04

--------------------
src_case_dir = ./dataset/prostate/Training/case05
dst_case_dir = ./dataset/working_data/Training/case05

--------------------
src_case_dir = ./dataset/prostate/Training/case06
dst_case_dir = ./dataset/working_data/Training/case06

--------------------
src_case_dir = ./dataset/prostate/Training/case07
dst_case_dir = ./dataset/working_data/Training/case07

--------------------
src_case_dir = ./dataset/prostate/Training/case08
dst_case_dir = ./dataset/working_data/Training/case08

-------------